In [1]:
pip install scikit-surprise

  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-macosx_10_9_x86_64.whl size=758105 sha256=9659322e303a588817e94733cfeffe0b614bc6a4abdf8bc44e326b14df08dcaf
  Stored in directory: /Users/yiqiwang/Library/Caches/pip/wheels/20/91/57/2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pandas as pd
import numpy as np
from surprise.model_selection import cross_validate

In [3]:
# import reviews_business_users_final to rbu_data
for line in open('reviews_business_users_final.json', 'r'):
    rbu_data = json.loads(line) 

In [6]:
# create a dataframe of reviews_business_users
df = pd.DataFrame.from_dict(rbu_data, orient='columns')

# preview df
pd.set_option('display.max_columns', None)
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,b_name,address,city,state,postal_code,latitude,longitude,b_stars,b_review_count,is_open,attributes,categories,hours,u_review_count,yelping_since,u_useful,u_funny,u_cool,elite,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,P320Yt8vFD3yjI34hq-umg,IMfkbLVt_GJfD7zJ9T4fcw,VPqWLp9kMiZEbctCebIZUA,3.0,0,0,0,"Overall is good, is worth the line up I don't ...",2018-09-25 03:22:50,Medina Cafe,780 Richards Street,Vancouver,BC,V6B 3A4,49.280443,-123.117036,4.0,2302,1,"{'GoodForKids': 'True', 'RestaurantsPriceRange...","Bars, Moroccan, Wine Bars, Breakfast & Brunch,...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-15:0', '...",12,2016-05-16 18:49:40,6,1,1,,0,4.67,0,0,0,0,0,0,0,0,0,0,0
1,d1GHSazhBfFsrcIjhlduHA,p4NHAQZS4EdyXwccRJFIrQ,VvmKYyGo4CDQ3rr9KMc4jg,5.0,0,0,0,I had so much fun walking up and down the aisl...,2018-03-23 13:46:42,Miniso,550 Granville Street,Vancouver,BC,V6C 1X6,49.283985,-123.115402,4.0,21,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Department Stores, Fashion, Shopping, Accessor...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",15,2017-02-11 21:09:01,2,1,1,,1,4.00,0,0,0,0,0,0,0,0,0,0,0
2,N5MsO0w9NWFderdSm1bEiQ,79cm11VzsOU9l3DW0lyYQw,VzBw4yBOaPc_Zniovd37yQ,2.0,0,1,0,This virtual ride takes you on a scenic and se...,2015-09-23 23:25:02,FlyOver Canada,"999 Canada Place, Unit 201",Vancouver,BC,V6C 3E1,49.289241,-123.108994,4.0,197,1,"{'BusinessParking': '{'garage': True, 'street'...","Cinema, Amusement Parks, Active Life, Arts & E...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",12,2013-10-04 21:24:44,17,1,3,,0,2.40,0,0,0,0,0,0,0,0,0,0,0
3,fH8hF8bOFGxITdwuvNQVkw,XK_VJRqf_91vCnZigJGfxw,IYyYpXSsD41g14_z-Zdtiw,5.0,0,0,0,"I found this Thai restaurant at yelp! \nSmall,...",2017-02-19 06:31:42,Kin Kao Thai Kitchen,903 Commercial Drive,Vancouver,BC,V5L,49.276356,-123.069692,4.0,143,1,"{'RestaurantsTakeOut': 'True', 'GoodForKids': ...","Thai, Restaurants","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'...",10,2015-05-25 00:56:03,16,7,7,,0,4.70,0,0,0,0,0,0,0,0,0,0,0
4,XPjCvOzRsoa-5-Dp3SNrjw,fNTuJcjClg4DXI_J_IcQpQ,KW2svfVHffW_FD8oeJo1xQ,1.0,0,0,0,Apparently it's a requirement to have a TERRIB...,2016-07-03 18:40:42,Something Healthy Wholesome Meals & Juice Bar,660 Abbott Street,Vancouver,BC,V6B 6N7,49.279138,-123.107450,3.5,34,0,"{'RestaurantsAttire': 'u'casual'', 'GoodForKid...","Sandwiches, Restaurants, Juice Bars & Smoothie...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",3,2016-07-03 18:26:57,6,0,1,,0,2.33,0,0,0,0,0,0,0,0,0,0,0


In [7]:
# check data shape
df.shape

(168738, 41)

## Collaborative Filtering (SVD, SVD++, KNNBasic)

In [18]:
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import SVD
from surprise.model_selection import train_test_split

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars']], reader)

# We'll use the famous SVD algorithm.
algo = SVD()

cross_validate(algo, data, measures=['RMSE', 'MAE'], verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3848  1.3793  1.3856  1.3745  1.3784  1.3805  0.0042  
MAE (testset)     1.1521  1.1480  1.1522  1.1455  1.1483  1.1492  0.0026  
Fit time          7.34    7.73    7.42    7.46    7.41    7.47    0.14    
Test time         0.20    0.21    0.21    0.66    0.63    0.38    0.21    


{'test_rmse': array([1.3848267 , 1.37929322, 1.3856392 , 1.37454307, 1.37837044]),
 'test_mae': array([1.1521488 , 1.14798166, 1.15223661, 1.14549048, 1.14831424]),
 'fit_time': (7.335147142410278,
  7.733577013015747,
  7.421915292739868,
  7.458085060119629,
  7.413188219070435),
 'test_time': (0.20377898216247559,
  0.21001076698303223,
  0.21468186378479004,
  0.6566648483276367,
  0.6300160884857178)}

In [19]:
from surprise import SVDpp

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars']], reader)

# We'll use the famous SVD algorithm.
algo = SVDpp()

cross_validate(algo, data, measures=['RMSE', 'MAE'], verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3845  1.3858  1.3898  1.3888  1.3902  1.3878  0.0023  
MAE (testset)     1.1540  1.1552  1.1598  1.1551  1.1592  1.1567  0.0023  
Fit time          19.21   20.07   18.87   20.22   20.04   19.68   0.54    
Test time         0.91    0.48    0.49    0.57    0.50    0.59    0.16    


{'test_rmse': array([1.38451601, 1.3858135 , 1.38981383, 1.38882557, 1.39019655]),
 'test_mae': array([1.15397896, 1.15521224, 1.15976623, 1.15514473, 1.15916779]),
 'fit_time': (19.208897829055786,
  20.06675887107849,
  18.86677885055542,
  20.224466800689697,
  20.03728199005127),
 'test_time': (0.905468225479126,
  0.48177313804626465,
  0.49359631538391113,
  0.5717310905456543,
  0.5049059391021729)}

In [20]:
from surprise import KNNBasic

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars']], reader)

# We'll use the famous SVD algorithm.
algo = KNNBasic()

cross_validate(algo, data, measures=['RMSE', 'MAE'], verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5771  1.5910  1.5744  1.5758  1.5831  1.5803  0.0061  
MAE (testset)     1.3300  1.3400  1.3247  1.3287  1.3352  1.3317  0.0053  
Fit time          229.98  261.43  298.87  210.77  243.78  248.96  29.97   
Test time         6.30    6.25    10.39   9.11    12.02   8.81    2.27    


{'test_rmse': array([1.57705312, 1.59095363, 1.57437741, 1.57584204, 1.58313258]),
 'test_mae': array([1.33002689, 1.34001171, 1.32469162, 1.32873587, 1.33517001]),
 'fit_time': (229.9765908718109,
  261.42576694488525,
  298.8696701526642,
  210.76548790931702,
  243.7756941318512),
 'test_time': (6.29994010925293,
  6.252881765365601,
  10.394863843917847,
  9.108435869216919,
  12.016791105270386)}

## Content Based Filtering (Cosine Similarity)